In [1]:
import sys
sys.path.append('../src/')

from benchmark.tools import *
from benchmark.models import Accuracy, Face, Box, Prediction, ImageFaces
from benchmark.utils import TsvHandle, load_aflw, tf_to_image_faces, get_box_id, run_detection
from benchmark.stats import predictions_summary

import wider_dataset
from facenet_pytorch import MTCNN

In [2]:
train, test, validation = tfds.load('wider_dataset', split=['train', 'test', 'validation'])

In [3]:
data = train.concatenate(test).concatenate(validation)

In [4]:
total_data = 3000

sys.stdout.write("Running detection, total samples: %d [sample/%d]" % (total_data, total_data))

Running detection, total samples: 3000 [sample/3000]

In [5]:
mtcnn = MTCNN(margin=14,
    factor=0.6,
    keep_all=True)

In [6]:
detector = lambda image: mtcnn.detect(image)[0]

In [7]:
with TsvHandle('mtcnn') as tsv:
    run_detection(tsv, validation.take(50), detector, False, False)

Running detection, total samples: 50
[====================] 100% - Sample: 50/50